In [46]:
import pyvisa
import time

rm = pyvisa.ResourceManager()
print(rm.list_resources())

('USB0::0xF4ED::0xEE3A::SDG08CBX7R0713::INSTR', 'USB0::0xF4ED::0xEE3A::SDG08CBX5R0530::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR')


In [ ]:
rm = pyvisa.ResourceManager()
inst1 = rm.open_resource(rm.list_resources()[0])  # Use your exact address
inst2 = rm.open_resource(rm.list_resources()[1])
print(inst1.query("*IDN?"))
# Set frequency and amplitude if needed

inst1.write('C1:BSWV FRQ,3;C1:BSWV PHSE,00;C1:BSWV AMP,3')#inst1.write('C1:BSWV AMP,2')          # 2Vpp
inst2.write('C1:BSWV FRQ,3;C1:BSWV PHSE,90;C1:BSWV AMP,3')        # 1kHz


# Change phase dynamically
#for phase in range(0, 360, 10):  # from 0 to 350 degrees
#    inst1.write(f'C1:BSWV PHSE,{phase}')
#    time.sleep(1)  # delay between updates

inst1.close()
inst2.close()
rm.close()

*IDN SDG,SDG830,SDG08CBX7R0713,1.08.01.15R3,08-00-00-13-00



In [48]:
boven = 'USB0::0xF4ED::0xEE3A::SDG08CBX7R0713::INSTR'